In [37]:
from __future__ import print_function

import tensorflow as tf
import os

# Dataset Parameters - CHANGE HERE
MODE = 'folder' # or 'file', if you choose a plain text file (see above).
DATASET_PATH = 'E:\HolisticClassification\Dataset' # the dataset file or root folder path.
BATCH_SIZE = 32

# Image Parameters
N_CLASSES = 2 # CHANGE HERE, total number of classes
IMG_HEIGHT = 32 # CHANGE HERE, the image height to be resized to
IMG_WIDTH = 32 # CHANGE HERE, the image width to be resized to
CHANNELS = 3 # The 3 color channels, change to 1 if grayscale

In [38]:
def read_images(dataset_path = DATASET_PATH, mode = MODE, batch_size = BATCH_SIZE):
    imagepaths, labels = list(), list()
    if mode == 'folder':
        label = 0
        classes = [f.path for f in os.scandir(DATASET_PATH) if f.is_dir() ] 
        for c in classes:
            c_dir = os.path.join(dataset_path, c)
            walk = os.walk(c_dir).__next__()
            for sample in walk[2]:
                if sample.endswith('.bmp') or sample.endswith('.BMP'):
                    imagepaths.append(os.path.join(c_dir, sample))
                    labels.append(label)
            label += 1
    else:
        raise Exception("Unknown mode.")

    imagepaths = tf.convert_to_tensor(imagepaths, dtype=tf.string)
    labels = tf.convert_to_tensor(labels, dtype=tf.int32)
    # Build a TF Queue, shuffle data
    image, label = tf.train.slice_input_producer([imagepaths, labels],
                                                 shuffle=True)

    # Read images from disk
    image = tf.read_file(image)
    image = tf.image.decode_bmp(image, channels=CHANNELS)

    # Resize images to a common size
    image = tf.image.resize_images(image, [IMG_HEIGHT, IMG_WIDTH])

    # Normalize
    image = image * 1.0/127.5 - 1.0

    # Create batches
    X, Y = tf.train.batch([image, label], batch_size=batch_size, capacity=batch_size * 8, num_threads=4)
    return X, Y

In [42]:
X_Train, Y_Train = read_images()
X_Train.shape
Y_Train.shape

TensorShape([Dimension(32)])